<a href="https://colab.research.google.com/github/isabellehaga/aq-data-scientist-test/blob/master/01_etl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Instruções:
        1- Download dos dados municipais do estado de Minas Gerais;
        2- Transformação dos dados de desmatamento;
        3- Processamento dos dados:
            -reprojeção para EPSG:31983;
            -cálculo de área;
        4- Geração dos arquivos de saída no formato GeoJSON;


# Análise Geoespacial do Desmatamento em Minas Gerais — Processo Seletivo para Cientista de Dados na AgronomiQ

# Tarefa 1

## Importando as Bibliotecas

In [2]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import seaborn as sns
from shapely.geometry import Point, Polygon


## Explorando e entendendo os dados

1- Download dos dados municipais do estado de Minas Gerais;

*📌 Observação: Como a fonte de dados está disponível em link público, não há necessidade de adicionar nenhum arquivo a este projeto para reproduzir os resultados.*

In [3]:
#Importando os dados
url = "https://raw.githubusercontent.com/tbrugz/geodata-br/master/geojson/geojs-31-mun.json"
gdf = gpd.read_file(url)

#Explorando e entendendo os dados
gdf.head()

,id,name,description,geometry
0,3100104,Abadia dos Dourados,Abadia dos Dourados,"POLYGON ((-47.4441 -18.15847, -47.43543 -18.16..."
1,3100203,Abaeté,Abaeté,"POLYGON ((-45.182 -18.88757, -45.17314 -18.889..."
2,3100302,Abre Campo,Abre Campo,"POLYGON ((-42.36955 -20.13218, -42.35755 -20.1..."
3,3100401,Acaiaca,Acaiaca,"POLYGON ((-43.11391 -20.35784, -43.10969 -20.3..."
4,3100500,Açucena,Açucena,"POLYGON ((-42.38592 -18.92388, -42.3814 -18.92..."


In [4]:
#Verificar duplicidade e dado ausente
gdf.describe()

,id,name,description,geometry
count,853,853,853,853
unique,853,853,853,853
top,3172202,Wenceslau Braz,Wenceslau Braz,"POLYGON ((-42.3540289472 -17.7833229926, -42.3..."
freq,1,1,1,1


In [5]:
# Verificar tipo de dados e confirmar número de linhas e se há dado faltante
gdf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 853 entries, 0 to 852
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   id           853 non-null    object  
 1   name         853 non-null    object  
 2   description  853 non-null    object  
 3   geometry     853 non-null    geometry
dtypes: geometry(1), object(3)
memory usage: 26.8+ KB


In [6]:
# Confirmar novamente se há duplicidade
gdf.duplicated().sum()

np.int64(0)

##📑 Descrição do Dataset

*   id: identificador único de cada município
*   name: nome do município
*   description: repetição do nome do município
*   geometry: polígono de delimitação do município

O dataset contém 853 linhas, o que corresponde exatamente ao número total de municípios do estado de Minas Gerais, conforme a lista oficial da Wikipédia (fonte: https://pt.wikipedia.org/wiki/Lista_de_munic%C3%ADpios_de_Minas_Gerais)

Além disso, o dataset não apresenta valores ausentes nem duplicados, garantindo a consistência inicial dos dados para as próximas etapas da análise.





## Transformação dos dados

Objetivos da segunda etapa:
*   Reprojetar os dados para o sistema de coordenadas EPSG:31983
*   Adicionar uma nova coluna contendo a área de cada município em km²
*   Salvar o arquivo no diretório dados/ com o nome municipios-mg.geojson
*   Converter o dataset para o formato GeoJSON


In [7]:
#2- Transformação dos dados de desmatamento;
# Reprojetar para EPSG:31983
gdf_31983 = gdf.to_crs(epsg=31983)

In [8]:
#3- Processamento dos dados:
 #       -reprojeção para EPSG:31983;
 #       -cálculo de área;
# No GeoDataFrame reprojetado, adicionamos uma nova coluna chamada "area_km2".
# Como o sistema de coordenadas EPSG:31983 usa metros, o cálculo da área retorna valores em m².
# Para obter a área em km², dividimos os valores por 1.000.000 (10**6).

gdf_31983["area_km2"] = gdf_31983.geometry.area / 10**6


In [9]:
# Verificando nova coluna :)
gdf_31983.head()

,id,name,description,geometry,area_km2
0,3100104,Abadia dos Dourados,Abadia dos Dourados,"POLYGON ((241429.802 7990562.876, 242359.697 7...",895.115178
1,3100203,Abaeté,Abaeté,"POLYGON ((480831.209 7911603.048, 481764.623 7...",1814.504046
2,3100302,Abre Campo,Abre Campo,"POLYGON ((775002.321 7771717.815, 776244.59 77...",472.049598
3,3100401,Acaiaca,Acaiaca,"POLYGON ((696872.185 7747792.052, 697305.64 77...",100.723996
4,3100500,Açucena,Açucena,"POLYGON ((775332.208 7905556.756, 775804.655 7...",812.875627


🧪 Teste extra de validação (opcional, mas recomendado)

Para garantir que o cálculo da área está correto, vamos comparar a área calculada pelo GeoDataFrame com a área oficial do município.

*   Erro absoluto (EA): diferença entre o valor calculado e o valor oficial.
*   Erro relativo (ER): erro absoluto dividido pelo valor oficial, mostrando a diferença proporcionalmente.

Esse teste é completamente opcional, mas fornece confiança na acurácia dos dados antes de prosseguir para as próximas etapas.

In [10]:
# Valores conhecidos
area_oficial = 880.461  # km², fonte: https://pt.wikipedia.org/wiki/Abadia_dos_Dourados
area_calculada = 895.115178  # km², calculada pelo GeoDataFrame

# Cálculo do erro absoluto
erro_absoluto = area_calculada - area_oficial

# Cálculo do erro relativo em %
erro_relativo = (erro_absoluto / area_oficial) * 100

# Exibição dos resultados
print(f"Erro absoluto: {erro_absoluto:.3f} km²")
print(f"Erro relativo: {erro_relativo:.2f}%")

# Interpretação:
# Valores de erro baixos indicam que o método de cálculo da área é confiável.
# Essa metodologia pode ser aplicada a outros municípios, estados ou dados geográficos de outros países.

Erro absoluto: 14.654 km²
Erro relativo: 1.66%


4- Geração dos arquivos de saída no formato GeoJSON;

Em vez de criar manualmente uma pasta dados e salvar o arquivo, optei por automatizar o processo, facilitando a reprodução dos resultados pelos recrutadores.


In [11]:
import os

pasta = "dados"

# Cria a pasta apenas se não existir
if not os.path.exists(pasta):
    os.makedirs(pasta)
    print(f"Pasta '{pasta}' criada com sucesso!")
else:
    print(f"Pasta '{pasta}' já existe.")

# Salvar o GeoJSON na pasta já existente
gdf_31983.to_file("dados/municipios-mg.geojson", driver="GeoJSON")

#Checar ao lado se o arquivo foi criado na pasta dados/

Pasta 'dados' criada com sucesso!


#Tarefa 2

Objetivos da etapa de dados socioeconômicos

*   Procurar uma fonte de dados confiável na internet para população e PIB dos municípios brasileiros.
*   Filtrar os dados para incluir apenas os municípios de Minas Gerais.
*   Salvar os dados em um arquivo CSV com o nome:
dados/populacao-pib-municipios-mg.csv.

Observação: Não é necessário refazer esses passos, pois o arquivo populacao-pib-municipios-mg.csv já foi gerado. O código completo foi mantido para fins didáticos e para demonstrar como os dados foram manipulados e filtrados.

##Filtrando os dados de PIB

A fonte dos dados de PIB e população é o site do IBGE: https://www.ibge.gov.br/

Nesta etapa, é necessário adicionar os arquivos no notebook. Na barra lateral à esquerda clicar no ícone icone.png
e adicionar os arquivos:


*   PIB dos Municípios - base de dados 2010-2021
*   pop_2024

ps: pode ser que demore por conta do tamanho do arquivo e da velocidade da sua internet.



In [12]:
# Importando o arquivo de PIB
arquivo1 = "PIB dos Municípios - base de dados 2010-2021.xlsx"

# Lendo o arquivo e transformando em DataFrame
df1 = pd.read_excel(arquivo1)


 Observação: o DataFrame possui várias linhas e colunas,
 mas para esta análise, nos interessam apenas as seguintes linhas e colunas:
   - Sigla da Unidade da Federação (MG)
   - Ano (2021)
   - Nome do Município
   - Produto Interno Bruto, a preços correntes (R$ 1.000)


In [13]:
# Criando um novo DataFrame contendo apenas os valores de Minas Gerais no ano de 2021
df_mg_2021 = df1[(df1['Sigla da Unidade da Federação'] == 'MG') & (df1['Ano'] == 2021)]

# Verificando se o filtro funcionou corretamente
df_mg_2021.head(5)

,Ano,Código da Grande Região,Nome da Grande Região,Código da Unidade da Federação,Sigla da Unidade da Federação,Nome da Unidade da Federação,Código do Município,Nome do Município,Região Metropolitana,Código da Mesorregião,...,"Valor adicionado bruto da Indústria,\na preços correntes\n(R$ 1.000)","Valor adicionado bruto dos Serviços,\na preços correntes \n- exceto Administração, defesa, educação e saúde públicas e seguridade social\n(R$ 1.000)","Valor adicionado bruto da Administração, defesa, educação e saúde públicas e seguridade social, \na preços correntes\n(R$ 1.000)","Valor adicionado bruto total, \na preços correntes\n(R$ 1.000)","Impostos, líquidos de subsídios, sobre produtos, \na preços correntes\n(R$ 1.000)","Produto Interno Bruto, \na preços correntes\n(R$ 1.000)","Produto Interno Bruto per capita, \na preços correntes\n(R$ 1,00)",Atividade com maior valor adicionado bruto,Atividade com segundo maior valor adicionado bruto,Atividade com terceiro maior valor adicionado bruto
63499,2021,3,Sudeste,31,MG,Minas Gerais,3100104,Abadia dos Dourados,NaN,3105,...,10618.285,43394.459,38588.616,167711.936,9061.722,176773.658,25174.26,"Pecuária, inclusive apoio à pecuária","Administração, defesa, educação e saúde públic...",Demais serviços
63500,2021,3,Sudeste,31,MG,Minas Gerais,3100203,Abaeté,NaN,3106,...,40612.969,220438.549,108971.880,497057.273,44181.933,541239.206,23266.10,Demais serviços,"Administração, defesa, educação e saúde públic...","Pecuária, inclusive apoio à pecuária"
63501,2021,3,Sudeste,31,MG,Minas Gerais,3100302,Abre Campo,NaN,3112,...,14020.634,100230.035,61425.579,242427.859,20258.680,262686.538,19553.86,Demais serviços,"Administração, defesa, educação e saúde públic...","Agricultura, inclusive apoio à agricultura e a..."
63502,2021,3,Sudeste,31,MG,Minas Gerais,3100401,Acaiaca,NaN,3112,...,12308.966,22209.014,23347.179,61428.753,3478.266,64907.018,16247.06,"Administração, defesa, educação e saúde públic...",Demais serviços,Indústrias extrativas
63503,2021,3,Sudeste,31,MG,Minas Gerais,3100500,Açucena,Colar Metropolitano,3108,...,75038.234,24519.318,41345.006,162867.106,2956.520,165823.625,17888.20,"Eletricidade e gás, água, esgoto, atividades d...","Administração, defesa, educação e saúde públic...",Demais serviços


In [14]:
## 2ª verificação do filtro :)
df_mg_2021[['Sigla da Unidade da Federação', 'Ano']].value_counts()


,,count
Sigla da Unidade da Federação,Ano,
MG,2021,853


In [15]:
# As colunas de interesse neste arquivo são:
# - "Produto Interno Bruto, a preços correntes (R$ 1.000)"
# - "Nome do Município"
# Por isso, estamos renomeando essas colunas para o arquivo final que será salvo.


df_mg_2021 = df_mg_2021.rename(columns={
    'Nome do Município': 'municipio',
    'Produto Interno Bruto, \na preços correntes\n(R$ 1.000)': 'PIB'
})

# Verificar se renomeou corretamente :)
df_mg_2021.head()


,Ano,Código da Grande Região,Nome da Grande Região,Código da Unidade da Federação,Sigla da Unidade da Federação,Nome da Unidade da Federação,Código do Município,municipio,Região Metropolitana,Código da Mesorregião,...,"Valor adicionado bruto da Indústria,\na preços correntes\n(R$ 1.000)","Valor adicionado bruto dos Serviços,\na preços correntes \n- exceto Administração, defesa, educação e saúde públicas e seguridade social\n(R$ 1.000)","Valor adicionado bruto da Administração, defesa, educação e saúde públicas e seguridade social, \na preços correntes\n(R$ 1.000)","Valor adicionado bruto total, \na preços correntes\n(R$ 1.000)","Impostos, líquidos de subsídios, sobre produtos, \na preços correntes\n(R$ 1.000)",PIB,"Produto Interno Bruto per capita, \na preços correntes\n(R$ 1,00)",Atividade com maior valor adicionado bruto,Atividade com segundo maior valor adicionado bruto,Atividade com terceiro maior valor adicionado bruto
63499,2021,3,Sudeste,31,MG,Minas Gerais,3100104,Abadia dos Dourados,NaN,3105,...,10618.285,43394.459,38588.616,167711.936,9061.722,176773.658,25174.26,"Pecuária, inclusive apoio à pecuária","Administração, defesa, educação e saúde públic...",Demais serviços
63500,2021,3,Sudeste,31,MG,Minas Gerais,3100203,Abaeté,NaN,3106,...,40612.969,220438.549,108971.880,497057.273,44181.933,541239.206,23266.10,Demais serviços,"Administração, defesa, educação e saúde públic...","Pecuária, inclusive apoio à pecuária"
63501,2021,3,Sudeste,31,MG,Minas Gerais,3100302,Abre Campo,NaN,3112,...,14020.634,100230.035,61425.579,242427.859,20258.680,262686.538,19553.86,Demais serviços,"Administração, defesa, educação e saúde públic...","Agricultura, inclusive apoio à agricultura e a..."
63502,2021,3,Sudeste,31,MG,Minas Gerais,3100401,Acaiaca,NaN,3112,...,12308.966,22209.014,23347.179,61428.753,3478.266,64907.018,16247.06,"Administração, defesa, educação e saúde públic...",Demais serviços,Indústrias extrativas
63503,2021,3,Sudeste,31,MG,Minas Gerais,3100500,Açucena,Colar Metropolitano,3108,...,75038.234,24519.318,41345.006,162867.106,2956.520,165823.625,17888.20,"Eletricidade e gás, água, esgoto, atividades d...","Administração, defesa, educação e saúde públic...",Demais serviços


In [16]:
# Criar um novo DataFrame apenas com as colunas 'municipio' e 'PIB'
# Isso facilita salvar os dados finais em CSV para uso posterior
df_PIB_final = df_mg_2021[['municipio', 'PIB']].copy()

# Visualizar os primeiros registros para conferência
df_PIB_final.head()

,municipio,PIB
63499,Abadia dos Dourados,176773.658
63500,Abaeté,541239.206
63501,Abre Campo,262686.538
63502,Acaiaca,64907.018
63503,Açucena,165823.625


In [17]:
# Resetar o índice do DataFrame para remover o índice antigo
df_PIB_final = df_PIB_final.reset_index(drop=True)

In [18]:
#somente para checar :)
df_PIB_final.head()

,municipio,PIB
0,Abadia dos Dourados,176773.658
1,Abaeté,541239.206
2,Abre Campo,262686.538
3,Acaiaca,64907.018
4,Açucena,165823.625


In [29]:
df_PIB_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 853 entries, 0 to 852
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   municipio  853 non-null    object 
 1   PIB        853 non-null    float64
dtypes: float64(1), object(1)
memory usage: 13.5+ KB


##Agora que temos os valores de PIB de cada município, vamos extrair os dados de população e, em seguida, unir ambos em um único arquivo CSV para análise.

DF final: df_PIB_final

##Filtando os dados de população

In [19]:
#Importando o arquivo
arquivo2 = "pop_2024.xls"

# Lendo o Excel e usando a segunda linha como header
df2 = pd.read_excel(arquivo2, sheet_name=1, header=1)

 Observação: o DataFrame possui várias linhas e colunas,
 mas para esta análise, nos interessam apenas as seguintes colunas:
   - UF (MG)
   - Nome do Município
   - População Estimada


In [20]:
df2.head()

,UF,COD. UF,COD. MUNIC,NOME DO MUNICÍPIO,POPULAÇÃO ESTIMADA,Unnamed: 5
0,RO,11.0,15.0,Alta Floresta D'Oeste,22853.0,NaN
1,RO,11.0,23.0,Ariquemes,108573.0,NaN
2,RO,11.0,31.0,Cabixi,5690.0,NaN
3,RO,11.0,49.0,Cacoal,97637.0,NaN
4,RO,11.0,56.0,Cerejeiras,16975.0,NaN


In [21]:
# Filtrar apenas Minas Gerais
df_mg_2024 = df2[df2['UF'] == 'MG']

In [22]:
# Verificar resultado
df_mg_2024.head()

,UF,COD. UF,COD. MUNIC,NOME DO MUNICÍPIO,POPULAÇÃO ESTIMADA,Unnamed: 5
2244,MG,31.0,104.0,Abadia dos Dourados,6365.0,NaN
2245,MG,31.0,203.0,Abaeté,23161.0,NaN
2246,MG,31.0,302.0,Abre Campo,14312.0,NaN
2247,MG,31.0,401.0,Acaiaca,3997.0,NaN
2248,MG,31.0,500.0,Açucena,8995.0,NaN


In [23]:
#2ª verificação
df_mg_2024['UF'].unique()

array(['MG'], dtype=object)

In [24]:
#Verificar se está faltando algum município
df_mg_2024.info()

<class 'pandas.core.frame.DataFrame'>
Index: 853 entries, 2244 to 3096
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   UF                  853 non-null    object 
 1   COD. UF             853 non-null    float64
 2   COD. MUNIC          853 non-null    float64
 3   NOME DO MUNICÍPIO   853 non-null    object 
 4   POPULAÇÃO ESTIMADA  853 non-null    float64
 5   Unnamed: 5          0 non-null      object 
dtypes: float64(3), object(3)
memory usage: 78.9+ KB


In [25]:
df_mg_2024 = df_mg_2024.rename(columns={
    'NOME DO MUNICÍPIO': 'municipio',
    'POPULAÇÃO ESTIMADA': 'populacao'
})

# Verificar resultado
df_mg_2024.head()

,UF,COD. UF,COD. MUNIC,municipio,populacao,Unnamed: 5
2244,MG,31.0,104.0,Abadia dos Dourados,6365.0,NaN
2245,MG,31.0,203.0,Abaeté,23161.0,NaN
2246,MG,31.0,302.0,Abre Campo,14312.0,NaN
2247,MG,31.0,401.0,Acaiaca,3997.0,NaN
2248,MG,31.0,500.0,Açucena,8995.0,NaN


In [26]:
# Criar um novo DataFrame apenas com as colunas 'municipio' e 'populacao'
# Isso facilita salvar os dados finais em CSV para uso posterior
df_mg_pop = df_mg_2024[['municipio', 'populacao']].copy()

# Verificar resultado
df_mg_pop.head()


,municipio,populacao
2244,Abadia dos Dourados,6365.0
2245,Abaeté,23161.0
2246,Abre Campo,14312.0
2247,Acaiaca,3997.0
2248,Açucena,8995.0


##Agora que temos o número de habitantes de cada município de Minas Gerais, vamos juntar com o DF anterior e unir ambos em um único arquivo CSV para análise.

DF final: df_mg_pop

## Resultado final


In [27]:
df_mg_pop

,municipio,populacao
2244,Abadia dos Dourados,6365.0
2245,Abaeté,23161.0
2246,Abre Campo,14312.0
2247,Acaiaca,3997.0
2248,Açucena,8995.0
...,...,...
3092,Virginópolis,10519.0
3093,Virgolândia,4530.0
3094,Visconde do Rio Branco,40608.0
3095,Volta Grande,4473.0


In [28]:
df_PIB_final

,municipio,PIB
0,Abadia dos Dourados,176773.658
1,Abaeté,541239.206
2,Abre Campo,262686.538
3,Acaiaca,64907.018
4,Açucena,165823.625
...,...,...
848,Virginópolis,183952.719
849,Virgolândia,62673.559
850,Visconde do Rio Branco,1351678.582
851,Volta Grande,140360.687


In [30]:
# Unir população + PIB pelo município
df_mg_completo = pd.merge(
    df_mg_pop,    # DataFrame com população
    df_PIB_final,    # DataFrame com PIB
    on='municipio',  # chave de junção
    how='inner'      # mantém apenas municípios que existem em ambos
)



In [31]:
# Verificar resultado
df_mg_completo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 853 entries, 0 to 852
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   municipio  853 non-null    object 
 1   populacao  853 non-null    float64
 2   PIB        853 non-null    float64
dtypes: float64(2), object(1)
memory usage: 20.1+ KB


In [32]:
# Salvar o DataFrame final em CSV na pasta 'dados'
# O arquivo gerado será: dados/populacao-pib-municipios-mg.csv
df_mg_completo.to_csv("dados/populacao-pib-municipios-mg.csv", index=False)

# <-- Verificar se o arquivo foi salvo corretamente
# e realizar o download, pois arquivos salvos no Colab
# serão perdidos ao fechar a sessão.


# Tarefa 3

Objetivos da etapa de dados de desmatamento

- Unir os arquivos dados/desmatamento_ago22.gpkg e dados/desmatamento_set_22.gpkg.em um único dataset.
- Transformar o dataset combinado em formato GeoJSON com projeção EPSG:31983.
- Salvar o arquivo final em: dados/focos-desmatamento-mg.geojson.

Observação: Não é necessário refazer os passos anteriores, pois o arquivo final já foi gerado a partir do código mostrado. O código completo foi mantido para fins didáticos e para demonstrar como os dados foram manipulados e processados.

Novamente será necessário adicionar os arquivos no notebook. Na barra lateral à esquerda clicar no ícone icone.pngicone.png e adicionar os arquivos:

- desmatamento_ago22.gpkg
- desmatamento_set22.gpkg

In [38]:
# Importando arquivos
file_ago = "/content/desmatamento_ago22.gpkg"
file_set = "/content/desmatamento_set22.gpkg"

gdf_ago = gpd.read_file(file_ago)
gdf_set = gpd.read_file(file_set)

In [39]:
# Verificar se os dados foram importados corretamente
# e se correspondem ao arquivo esperado (desmatamento_ago22.gpkg),
# garantindo que as datas sejam referentes a agosto.
gdf_ago.head(4)

,ogc_fid,id,Instância,Data Inicial,Data Final,Cena,Score,Data Processamento,Bioma,NuFis,Cia PM,geometry
0,1,6726,9696,26/07/2022,05/08/2022,22KFC,0.9,08/08/2022 13:08,Cerrado,Triângulo Mineiro,5,"POLYGON ((-49.15074 -20.03366, -49.15028 -20.0..."
1,2,6727,9700,21/07/2022,26/07/2022,22KFD,0.9,08/08/2022 13:08,Cerrado,Triângulo Mineiro,5,"POLYGON ((-49.45834 -19.46058, -49.45555 -19.4..."
2,3,6728,9701,21/07/2022,26/07/2022,22KFD,0.9,08/08/2022 13:08,Cerrado,Triângulo Mineiro,5,"POLYGON ((-49.28326 -19.63463, -49.28261 -19.6..."
3,4,6729,9702,21/07/2022,05/08/2022,22KFD,0.93,08/08/2022 13:08,Cerrado,Triângulo Mineiro,5,"POLYGON ((-49.28382 -19.63454, -49.28289 -19.6..."


In [40]:
# Verificar se os dados foram importados corretamente
# e se correspondem ao arquivo esperado (desmatamento_set22.gpkg),
# garantindo que as datas sejam referentes a setembro.
gdf_set.head()

,Instância,Data Inicial,Data Final,Cena,Score,Bioma,NuFis,Cia PM,id,geometry
0,489,27/08/2022,01/09/2022,22KHC,0.85,Mata Atlântica,Triângulo Mineiro,5,1,"POLYGON ((-47.88682 -20.0351, -47.8885 -20.035..."
1,451,04/09/2022,19/09/2022,22KGC,0.91,Cerrado,Triângulo Mineiro,5,2,"POLYGON ((-48.79622 -20.01484, -48.79743 -20.0..."
2,56,15/08/2022,09/09/2022,22KEC,0.94,Mata Atlântica,Triângulo Mineiro,5,3,"POLYGON ((-50.96237 -20.00151, -50.96284 -20.0..."
3,370,14/09/2022,24/09/2022,22KFD,0.91,Cerrado,Triângulo Mineiro,5,4,"POLYGON ((-49.05217 -19.94629, -49.05161 -19.9..."
4,618,06/09/2022,11/09/2022,22KHD,0.87,Mata Atlântica,Triângulo Mineiro,5,5,"POLYGON ((-47.84676 -19.94933, -47.84713 -19.9..."


In [41]:
# Concatenar os dois GeoDataFrames de desmatamento (agosto e setembro)
gdf_combined = gpd.GeoDataFrame(pd.concat([gdf_ago, gdf_set], ignore_index=True))

# Verificar o sistema de referência de coordenadas (CRS) atual
print("CRS atual:", gdf_combined.crs)

# Observação: mais adiante, será necessário converter o CRS para EPSG:31983


CRS atual: EPSG:4326


## ps: Somente para testar se a concatenação foi bem sucedida

In [44]:
# Verificando se a concatenação foi sucedida
# Número de linhas em cada GeoDataFrame
print("Número de linhas em gdf_ago:", gdf_ago.shape[0])
print("Número de linhas em gdf_set:", gdf_set.shape[0])
print("Número de linhas em gdf_combined:", gdf_combined.shape[0])


Número de linhas em gdf_ago: 617
Número de linhas em gdf_set: 1422
Número de linhas em gdf_combined: 2039


In [45]:
print(gdf_ago.shape[0] + gdf_set.shape[0] == gdf_combined.shape[0])

True


##

In [46]:
gdf_combined.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 2039 entries, 0 to 2038
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype   
---  ------              --------------  -----   
 0   ogc_fid             617 non-null    float64 
 1   id                  2039 non-null   int64   
 2   Instância           2039 non-null   int64   
 3   Data Inicial        2039 non-null   object  
 4   Data Final          2039 non-null   object  
 5   Cena                2039 non-null   object  
 6   Score               2039 non-null   object  
 7   Data Processamento  617 non-null    object  
 8   Bioma               2039 non-null   object  
 9   NuFis               2039 non-null   object  
 10  Cia PM              2039 non-null   object  
 11  geometry            2039 non-null   geometry
dtypes: float64(1), geometry(1), int64(2), object(8)
memory usage: 191.3+ KB


In [47]:
# Reprojetar para EPSG:31983
gdf_combined = gdf_combined.to_crs(epsg=31983)

# Salvar como GeoJSON
gdf_combined.to_file("dados/focos-desmatamento-mg.geojson", driver="GeoJSON")

# Agora salve todos os arquivos gerados até o momento (caso ainda não tenha feito). Eles serão utilizados no segundo notebook, então é importante que estejam disponíveis.